In [ ]:
chave_openai="<your_openai_key>"

In [ ]:
import os
import pickle
import pandas as pd
from transformers import GPT2TokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
with open('website_content_parsed.txt', 'r', encoding="utf-8") as f:
    text = f.read()

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=chave_openai, model="text-embedding-ada-002")

db = FAISS.from_documents(chunks, embeddings)

with open("embeddings.pkl", "wb") as f:
    pickle.dump(db, f)

In [ ]:
query = "What the Bard Class is good for?"
docs = db.similarity_search(query)
docs[0]

In [ ]:
chain = load_qa_chain(OpenAI(openai_api_key=chave_openai, temperature=0), chain_type="stuff")

chain.run(input_documents=docs, question=query)